In [240]:
import pandas as pd
import re
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV  # Linear least squares with l2 regularization
from L_model import LLS
#from split import train_test_split

In [241]:
data = pd.read_csv('HousePrice.csv')
data.head()

,Area,Room,Parking,Warehouse,Elevator,Address,Price,Price(USD)
0,63,1,True,True,True,Shahran,1.850000e+09,61666.67
1,60,1,True,True,True,Shahran,1.850000e+09,61666.67
2,79,2,True,True,True,Pardis,5.500000e+08,18333.33
3,95,2,True,True,True,Shahrake Qods,9.025000e+08,30083.33
4,123,2,True,True,True,Shahrake Gharb,7.000000e+09,233333.33


In [242]:
data.sample(10)

,Area,Room,Parking,Warehouse,Elevator,Address,Price,Price(USD)
1657,57,1,True,True,True,Feiz Garden,1.890000e+09,63000.00
1534,228,3,True,True,True,Zaferanieh,2.029200e+10,676400.00
2061,61,2,False,True,False,Salsabil,1.040000e+09,34666.67
867,74,2,True,True,True,Shahran,4.100000e+09,136666.67
2434,39,1,False,True,False,Salsabil,8.000000e+08,26666.67
2887,68,2,True,True,True,Ostad Moein,1.970000e+09,65666.67
2220,105,3,True,True,True,Shahr-e-Ziba,3.550000e+09,118333.33
367,77,2,True,True,True,Shahr-e-Ziba,2.570000e+09,85666.67
1875,75,2,False,True,True,Parand,4.000000e+08,13333.33
2391,140,3,True,True,True,Kamranieh,8.100000e+09,270000.00


In [243]:
data.columns

Index(['Area', 'Room', 'Parking', 'Warehouse', 'Elevator', 'Address', 'Price',
       'Price(USD)'],
      dtype='object')

In [244]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3479 entries, 0 to 3478
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Area        3479 non-null   object 
 1   Room        3479 non-null   int64  
 2   Parking     3479 non-null   bool   
 3   Warehouse   3479 non-null   bool   
 4   Elevator    3479 non-null   bool   
 5   Address     3456 non-null   object 
 6   Price       3479 non-null   float64
 7   Price(USD)  3479 non-null   float64
dtypes: bool(3), float64(2), int64(1), object(2)
memory usage: 146.2+ KB


In [245]:
data['Area'] = data['Area'].apply(lambda x: re.sub(',', '', x))
data['Area'] = pd.to_numeric(data['Area'] , errors='coerce')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3479 entries, 0 to 3478
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Area        3479 non-null   int64  
 1   Room        3479 non-null   int64  
 2   Parking     3479 non-null   bool   
 3   Warehouse   3479 non-null   bool   
 4   Elevator    3479 non-null   bool   
 5   Address     3456 non-null   object 
 6   Price       3479 non-null   float64
 7   Price(USD)  3479 non-null   float64
dtypes: bool(3), float64(2), int64(2), object(1)
memory usage: 146.2+ KB


In [246]:
data.shape

(3479, 8)

In [247]:
data.isnull().sum()

Area           0
Room           0
Parking        0
Warehouse      0
Elevator       0
Address       23
Price          0
Price(USD)     0
dtype: int64

In [248]:
data.dropna(inplace= True)
data.shape

(3456, 8)

In [249]:
boolean_features = ['Parking','Warehouse','Elevator']
data[boolean_features] = data[boolean_features].astype('Int64')
data.head(10)

,Area,Room,Parking,Warehouse,Elevator,Address,Price,Price(USD)
0,63,1,1,1,1,Shahran,1.850000e+09,61666.67
1,60,1,1,1,1,Shahran,1.850000e+09,61666.67
2,79,2,1,1,1,Pardis,5.500000e+08,18333.33
3,95,2,1,1,1,Shahrake Qods,9.025000e+08,30083.33
4,123,2,1,1,1,Shahrake Gharb,7.000000e+09,233333.33
5,70,2,1,1,0,North Program Organization,2.050000e+09,68333.33
6,87,2,1,1,1,Pardis,6.000000e+08,20000.00
7,59,1,1,1,1,Shahran,2.150000e+09,71666.67
8,54,2,1,1,0,Andisheh,4.930000e+08,16433.33
9,71,1,1,1,1,West Ferdows Boulevard,2.370000e+09,79000.00


In [250]:
data2 = data.copy()
data2.loc[:, "Price"] =data["Price"].map('{:,.0f}'.format)

In [251]:
data2

,Area,Room,Parking,Warehouse,Elevator,Address,Price,Price(USD)
0,63,1,1,1,1,Shahran,"1,850,000,000",61666.67
1,60,1,1,1,1,Shahran,"1,850,000,000",61666.67
2,79,2,1,1,1,Pardis,"550,000,000",18333.33
3,95,2,1,1,1,Shahrake Qods,"902,500,000",30083.33
4,123,2,1,1,1,Shahrake Gharb,"7,000,000,000",233333.33
...,...,...,...,...,...,...,...,...
3474,86,2,1,1,1,Southern Janatabad,"3,500,000,000",116666.67
3475,83,2,1,1,1,Niavaran,"6,800,000,000",226666.67
3476,75,2,0,0,0,Parand,"365,000,000",12166.67
3477,105,2,1,1,1,Dorous,"5,600,000,000",186666.67


In [252]:
data['Price(USD)'] = data['Price(USD)'].apply(lambda x : x * 0.6 )

data['Price(USD)']


0        37000.002
1        37000.002
2        10999.998
3        18049.998
4       139999.998
           ...    
3474     70000.002
3475    136000.002
3476      7300.002
3477    112000.002
3478      7200.000
Name: Price(USD), Length: 3456, dtype: float64

In [253]:
data.Address.unique()

array(['Shahran', 'Pardis', 'Shahrake Qods', 'Shahrake Gharb',
       'North Program Organization', 'Andisheh', 'West Ferdows Boulevard',
       'Narmak', 'Saadat Abad', 'Zafar', 'Islamshahr', 'Pirouzi',
       'Shahrake Shahid Bagheri', 'Moniriyeh', 'Velenjak', 'Amirieh',
       'Southern Janatabad', 'Salsabil', 'Zargandeh', 'Feiz Garden',
       'Water Organization', 'ShahrAra', 'Gisha', 'Ray', 'Abbasabad',
       'Ostad Moein', 'Farmanieh', 'Parand', 'Punak', 'Qasr-od-Dasht',
       'Aqdasieh', 'Pakdasht', 'Railway', 'Central Janatabad',
       'East Ferdows Boulevard', 'Pakdasht KhatunAbad', 'Sattarkhan',
       'Baghestan', 'Shahryar', 'Northern Janatabad', 'Daryan No',
       'Southern Program Organization', 'Rudhen', 'West Pars', 'Afsarieh',
       'Marzdaran', 'Dorous', 'Sadeghieh', 'Chahardangeh', 'Baqershahr',
       'Jeyhoon', 'Lavizan', 'Shams Abad', 'Fatemi',
       'Keshavarz Boulevard', 'Kahrizak', 'Qarchak',
       'Northren Jamalzadeh', 'Azarbaijan', 'Bahar',
       'P

In [254]:
len(data.Address.unique())

192

In [255]:
data_address = data['Address'].value_counts().copy()
data_address

Address
Punak                     161
Pardis                    146
West Ferdows Boulevard    145
Gheitarieh                141
Shahran                   130
                         ... 
Chardangeh                  1
Mehrabad                    1
Pakdasht KhatunAbad         1
Kazemabad                   1
Yakhchiabad                 1
Name: count, Length: 192, dtype: int64

In [256]:
data_address_counts = data['Address'].value_counts().reset_index()
data_address_counts.columns = ['Address', 'Counts']

fig = px.bar(data_address_counts, x='Address', y='Counts', title='Address Counts')
fig.show()

In [257]:
top_5 = data.sort_values('Price',ascending=False)[:5]
top_5

,Area,Room,Parking,Warehouse,Elevator,Address,Price,Price(USD)
1707,420,4,1,1,1,Zaferanieh,9.240000e+10,1848000.000
1810,705,5,1,1,0,Abazar,9.100000e+10,1819999.998
430,400,5,1,1,0,Lavasan,8.500000e+10,1699999.998
819,680,5,1,1,0,Ekhtiarieh,8.160000e+10,1632000.000
1332,350,4,1,1,1,Niavaran,8.050000e+10,1609999.998


In [258]:
top_5['Price(Millions USD)'] = top_5['Price(USD)'] / 1e6
df_sorted = top_5.sort_values('Price(USD)', ascending=False)
fig = px.bar(df_sorted, x='Address', y='Price(Millions USD)', title='The 5 most expensive houses',
             labels={'Price(Millions USD)': 'Price (Millions USD)', 'Address': 'Address'},
             hover_data={'Area': True, 'Price(USD)': ':.2e'},
             text='Area',
             )
fig.update_layout(xaxis_title='Address',
                  yaxis_title='Price (Millions USD)',
                  yaxis_type='log',
                  uniformtext_minsize=8,
                  )
fig.show()

In [259]:
desire_features = ['Area', 'Room','Parking', 'Warehouse', 'Elevator']
x_dataset = data[desire_features].values
x_dataset

array([[63, 1, 1, 1, 1],
       [60, 1, 1, 1, 1],
       [79, 2, 1, 1, 1],
       ...,
       [75, 2, 0, 0, 0],
       [105, 2, 1, 1, 1],
       [82, 2, 0, 1, 1]], dtype=object)

In [260]:
y_dataset = data['Price'].values
x_dataset = x_dataset.astype(np.float64)
y_dataset = y_dataset.astype(np.float64)
x_train,x_test,y_train,y_test = train_test_split(x_dataset, y_dataset, test_size= 0.2)


In [261]:
lss_model = LLS()
lss_model.fit(x_train,y_train)
Y_pred = lss_model.predict(x_test)

print(Y_pred)

[ 4.91912844e+09  2.89263168e+08  1.97769814e+10  4.91912843e+09
  5.88738554e+09  9.54899372e+09  1.25752027e+09  5.88738554e+09
  8.62976194e+09  4.91912844e+09  4.91912844e+09  5.88738554e+09
  4.91912844e+09  4.91912846e+09  4.91912844e+09  9.54899371e+09
  4.91912845e+09  4.91912845e+09  9.54899372e+09  2.89263165e+08
  4.91912844e+09  9.19231784e+08  1.51471161e+10  1.97769815e+10
  4.91912844e+09  4.91912845e+09  2.89263175e+08  5.88738553e+09
  9.54899374e+09  4.91912844e+09  4.91912844e+09  2.89263166e+08
  4.91912843e+09  4.91912843e+09  2.89263171e+08  4.58083994e+09
  9.54899374e+09  4.91912844e+09  1.25752027e+09  9.54899372e+09
  9.54899374e+09  1.41788591e+10  2.89263168e+08  4.62986528e+09
  4.91912844e+09  3.99989667e+09  9.54899378e+09  4.91912844e+09
  4.91912843e+09  4.91912843e+09  4.91912844e+09  2.89263172e+08
  4.91912843e+09  9.54899371e+09  4.91912844e+09  4.91912844e+09
  1.05172508e+10  9.54899373e+09  2.89263169e+08  9.19231779e+08
  1.25752027e+09  2.89263

In [262]:
# Calculate MAE
mae_custom = mean_absolute_error(y_test, Y_pred)
print("(MAE):", mae_custom)

# Calculate MSE
mse_custom = mean_squared_error(y_test, Y_pred)
print("(MSE):", mse_custom)

# Calculate RMSE
rmse_custom = np.sqrt(mse_custom)
print("(RMSE):", rmse_custom)


(MAE): 4236544297.051161
(MSE): 5.485155257846084e+19
(RMSE): 7406183401.621975


In [263]:
lr_model = LinearRegression()
lr_model.fit(x_train, y_train)
lr_predictions = lr_model.predict(x_test)
mae_lr = mean_absolute_error(y_test, lr_predictions)
mse_lr = mean_squared_error(y_test, lr_predictions)
rmse_lr = np.sqrt(mse_lr)

ridgecv_model = RidgeCV()
ridgecv_model.fit(x_train, y_train)
ridgecv_predictions = ridgecv_model.predict(x_test)
mae_ridgecv = mean_absolute_error(y_test, ridgecv_predictions)
mse_ridgecv = mean_squared_error(y_test, ridgecv_predictions)
rmse_ridgecv = np.sqrt(mse_ridgecv)

print("Mean Absolute Error (MAE):")
print("Custom LLS model:", mae_custom)
print("Linear Regression model:", mae_lr)
print("RidgeCV model:", mae_ridgecv)

print("\nMean Squared Error (MSE):")
print("Custom LLS model:", mse_custom)
print("Linear Regression model:", mse_lr)
print("RidgeCV model:", mse_ridgecv)

print("\nRoot Mean Squared Error (RMSE):")
print("Custom LLS model:", rmse_custom)
print("Linear Regression model:", rmse_lr)
print("RidgeCV model:", rmse_ridgecv)


Mean Absolute Error (MAE):
Custom LLS model: 4236544297.051161
Linear Regression model: 4175303072.2208385
RidgeCV model: 446878026848.237

Mean Squared Error (MSE):
Custom LLS model: 5.485155257846084e+19
Linear Regression model: 4.971978836447607e+19
RidgeCV model: 4.264594921573452e+25

Root Mean Squared Error (RMSE):
Custom LLS model: 7406183401.621975
Linear Regression model: 7051226018.535789
RidgeCV model: 6530386605380.612


In [264]:
models = ['Custom LLS', 'Linear Regression', 'RidgeCV']
mae_values = [mae_custom, mae_lr, mae_ridgecv]
mse_values = [mse_custom, mse_lr, mse_ridgecv]
rmse_values = [rmse_custom, rmse_lr, rmse_ridgecv]
fig = make_subplots(rows=1, cols=3, subplot_titles=['Mean Absolute Error (MAE)', 'Mean Squared Error (MSE)', 'Root Mean Squared Error (RMSE)'])
fig.add_trace(go.Bar(x=models, y=mae_values, name='MAE'), row=1, col=1)
fig.add_trace(go.Bar(x=models, y=mse_values, name='MSE'), row=1, col=2)
fig.add_trace(go.Bar(x=models, y=rmse_values, name='RMSE'), row=1, col=3)
fig.update_layout(title='MAE and MSE and RMSE Comparison',
                  xaxis=dict(title='Models'),
                  yaxis=dict(title='Error'),
                  template='plotly_white')
fig.show()


In [265]:
models = ['Custom LLS', 'Linear Regression', 'RidgeCV']
mae_values = [mae_custom, mae_lr]
mse_values = [mse_custom, mse_lr]
rmse_values = [rmse_custom, rmse_lr]
fig = make_subplots(rows=1, cols=3, subplot_titles=['Mean Absolute Error (MAE)', 'Mean Squared Error (MSE)', 'Root Mean Squared Error (RMSE)'])
fig.add_trace(go.Bar(x=models, y=mae_values, name='MAE'), row=1, col=1)
fig.add_trace(go.Bar(x=models, y=mse_values, name='MSE'), row=1, col=2)
fig.add_trace(go.Bar(x=models, y=rmse_values, name='RMSE'), row=1, col=3)
fig.update_layout(title='MAE and MSE and RMSE Comparison',
                  xaxis=dict(title='Models'),
                  yaxis=dict(title='Error'),
                  template='plotly_white')
fig.show()

In [266]:
# Punak:
Punak_dataset = data[data['Address'].map(lambda x :x == 'Punak')]
desire_features = ['Area', 'Room','Parking', 'Warehouse', 'Elevator']
x_dataset = Punak_dataset[desire_features].values
y_dataset = Punak_dataset['Price(USD)'].values
x_dataset = x_dataset.astype(np.float64)
y_dataset = y_dataset.astype(np.float64)
x_train,x_test,y_train,y_test = train_test_split(x_dataset, y_dataset, test_size= 0.2)
lls_model = LLS()
lls_model.fit(x_train,y_train)
Y_pred = lls_model.predict(x_test)


lr_model_p = LinearRegression()
lr_model_p.fit(x_train, y_train)
lr_pred_p = lr_model_p.predict(x_test)

ridgecv_model_p = RidgeCV()
ridgecv_model_p.fit(x_train, y_train)
ridgecv_pred_p = ridgecv_model_p.predict(x_test)


# Calculate MAE
mae_p = mean_absolute_error(y_test, Y_pred)
mae_lr = mean_absolute_error(y_test, lr_pred_p)
mae_ridgecv = mean_absolute_error(y_test, ridgecv_pred_p)

# Calculate MSE
mse_p = mean_squared_error(y_test, Y_pred)
mse_lr = mean_squared_error(y_test, lr_pred_p)
mse_ridgecv = mean_squared_error(y_test, ridgecv_pred_p)

# Calculate RMSE
rmse_p = np.sqrt(mse_p)
rmse_lr = np.sqrt(mse_lr)
rmse_ridgecv = np.sqrt(mse_ridgecv)

print("Mean Absolute Error (MAE):")
print("Custom LLS model:", mae_p)
print("Linear Regression model:", mae_lr)
print("RidgeCV model:", mae_p)

print("\nMean Squared Error (MSE):")
print("Custom LLS model:", mse_p)
print("Linear Regression model:", mse_lr)
print("RidgeCV model:", mse_p)

print("\nRoot Mean Squared Error (RMSE):")
print("Custom LLS model:", rmse_p)
print("Linear Regression model:", rmse_lr)
print("RidgeCV model:", rmse_ridgecv)


disp_custom = Y_pred[:10]
disp_lls = lr_pred_p[:10]

Mean Absolute Error (MAE):
Custom LLS model: 10530.482143264906
Linear Regression model: 9616.703208244162
RidgeCV model: 10530.482143264906

Mean Squared Error (MSE):
Custom LLS model: 299653337.3772041
Linear Regression model: 260731024.84270582
RidgeCV model: 299653337.3772041

Root Mean Squared Error (RMSE):
Custom LLS model: 17310.497895127224
Linear Regression model: 16147.167703430401
RidgeCV model: 16103.395069621307


In [293]:
actual_price = y_test[:10]
disp_custom = Y_pred[:10]
disp_lls = lr_pred_p[:10]
Houses=list(range(1,11))

disp_custom_text = [f'{val/1000:.2f}' for val in disp_custom]
disp_lls_text = [f'{val/1000:.2f}' for val in disp_lls]
actual_price_text = [f'{val/1000:.2f}' for val in actual_price]

fig = go.Figure(data=[
    go.Bar(name='Custom LLS', x=Houses,y= disp_custom, text = disp_custom_text),
    go.Bar(name='SKlearn LLS', x=Houses, y=disp_lls, text = disp_lls_text),
    go.Bar(name='Actual Prices', x=Houses, y=actual_price, text = actual_price_text)
])

fig.update_layout(title='Custom and SKlearn LLS prediction methods Comparison for 10 Houses',
                  barmode='group',
                  yaxis=dict(title='Price (k$)'),
                  xaxis=dict(title='House index'),
                  )
fig.show()

In [142]:

Houses


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]